##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
variable_name = "" # @param {"type":"string"}
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3siZmlsZV9kYXRhIjp7Im1pbWVfdHlwZSI6ImltYWdlL2pwZWciLCJ1cmwiOiJodHRwczovL3d3dy5nc3RhdGljLmNvbS9haXN0dWRpby9zYW1wbGUtaW1hZ2VzL2Nyb2lzc2FudC5qcGcifX0seyJ0ZXh0IjoiR2l2ZW4gdGhpcyBpbWFnZTpcXG5cXG4xLiBGaXJzdCwgZGVzY3JpYmUgdGhlIGltYWdlXFxuMi4gVGhlbiwgZGV0YWlsIHRoZSByZWNpcGUgdG8gYmFrZSB0aGlzIGl0ZW0gaW4gSlNPTiBmb3JtYXQuIEluY2x1ZGUgaXRlbSBuYW1lcyBhbmQgcXVhbnRpdGllcyBmb3IgdGhlIHJlY2lwZSJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUncyBhIHJlc3BvbnNlIGFkZHJlc3NpbmcgeW91ciByZXF1ZXN0czpcXG5cXG4qKjEuIEltYWdlIERlc2NyaXB0aW9uOioqXFxuXFxuVGhlIGltYWdlIHNob3dzIGEgY2xvc2UtdXAgdmlldyBvZiBhIGJhc2tldCBvdmVyZmxvd2luZyB3aXRoIGZyZXNobHkgYmFrZWQgY3JvaXNzYW50cy4gIFRoZSBjcm9pc3NhbnRzIGFyZSBnb2xkZW4gYnJvd24sIGV4aGliaXRpbmcgYSBmbGFreSwgbGF5ZXJlZCB0ZXh0dXJlLiBUaGV5IGFyZSBwaWxlZCBoYXBoYXphcmRseSwgY3JlYXRpbmcgYSB2aXN1YWxseSBhcHBlYWxpbmcgYW5kIGFwcGV0aXppbmcgYXJyYW5nZW1lbnQuICBUaGUgYmFza2V0IGFwcGVhcnMgdG8gYmUgbWFkZSBvZiBhIGxpZ2h0LWNvbG9yZWQgZmFicmljLiAgVGhlIGJhY2tncm91bmQgaXMgc2xpZ2h0bHkgYmx1cnJlZCBidXQgc2hvd3MgaGludHMgb2YgYWRkaXRpb25hbCBwYXN0cmllcywgc3VnZ2VzdGluZyBhIGJha2VyeSBzZXR0aW5nLiAgVGhlIG92ZXJhbGwgaW1wcmVzc2lvbiBpcyBvbmUgb2Ygd2FybXRoLCBhYnVuZGFuY2UsIGFuZCBkZWxpY2lvdXNuZXNzLlxcblxcblxcbioqMi4gQ3JvaXNzYW50IFJlY2lwZSBpbiBKU09OIEZvcm1hdDoqKlxcblxcblByb3ZpZGluZyBhIHByZWNpc2UgcmVjaXBlIGluIEpTT04gdGhhdCBndWFyYW50ZWVzIHRoZSBleGFjdCBhcHBlYXJhbmNlIG9mIHRoZSBjcm9pc3NhbnRzIGluIHRoZSBpbWFnZSBpcyBpbXBvc3NpYmxlLiAgVGhlIGxvb2sgb2YgYSBjcm9pc3NhbnQgZGVwZW5kcyBvbiBtYW55IGZhY3RvcnMgaW5jbHVkaW5nIG92ZW4gdHlwZSwgZG91Z2ggaGFuZGxpbmcsIGFuZCBpbmdyZWRpZW50IHF1YWxpdHkuICBIb3dldmVyLCBJIGNhbiBvZmZlciBhIEpTT04gcmVwcmVzZW50YXRpb24gb2YgYSByZWNpcGUgdGhhdCBwcm9kdWNlcyAqc2ltaWxhciogcmVzdWx0czpcXG5cXG5cXG5gYGBqc29uXFxue1xcbiAgXFxcInJlY2lwZU5hbWVcXFwiOiBcXFwiQ2xhc3NpYyBDcm9pc3NhbnRzXFxcIixcXG4gIFxcXCJ5aWVsZFxcXCI6IFxcXCIxMiBjcm9pc3NhbnRzXFxcIixcXG4gIFxcXCJwcmVwVGltZVxcXCI6IFxcXCIyIGhvdXJzIChwbHVzIHByb29maW5nIHRpbWUpXFxcIixcXG4gIFxcXCJiYWtlVGltZVxcXCI6IFxcXCIyMC0yNSBtaW51dGVzXFxcIixcXG4gIFxcXCJpbmdyZWRpZW50c1xcXCI6IFtcXG4gICAge1xcbiAgICAgIFxcXCJpdGVtXFxcIjogXFxcIkFsbC1wdXJwb3NlIGZsb3VyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiNTAwZ1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJpdGVtXFxcIjogXFxcIldhdGVyXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMzAwbWxcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwiaXRlbVxcXCI6IFxcXCJNaWxrXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiNTBtbFxcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJpdGVtXFxcIjogXFxcIlVuc2FsdGVkIGJ1dHRlclxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjI1MGcgKGZvciB0aGUgZG91Z2ggYW5kIGxhbWluYXRpb24pXFxcIlxcbiAgICB9LFxcbiAgICB7XFxuICAgICAgXFxcIml0ZW1cXFwiOiBcXFwiSW5zdGFudCBkcnkgeWVhc3RcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxMGdcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwiaXRlbVxcXCI6IFxcXCJTdWdhclxcXCIsXFxuICAgICAgXFxcInF1YW50aXR5XFxcIjogXFxcIjUwZ1xcXCJcXG4gICAgfSxcXG4gICAge1xcbiAgICAgIFxcXCJpdGVtXFxcIjogXFxcIlNhbHRcXFwiLFxcbiAgICAgIFxcXCJxdWFudGl0eVxcXCI6IFxcXCIxMGdcXFwiXFxuICAgIH0sXFxuICAgIHtcXG4gICAgICBcXFwiaXRlbVxcXCI6IFxcXCJFZ2cgd2FzaCAoZm9yIGdsYXppbmcpXFxcIixcXG4gICAgICBcXFwicXVhbnRpdHlcXFwiOiBcXFwiMSBlZ2csIGJlYXRlbiB3aXRoIDEgdGJzcCB3YXRlclxcXCJcXG4gICAgfVxcbiAgXSxcXG4gIFxcXCJpbnN0cnVjdGlvbnNcXFwiOiBbXFxuICAgIFxcXCJNYWtlIHRoZSBkb3VnaDogQ29tYmluZSBmbG91ciwgeWVhc3QsIHN1Z2FyLCBhbmQgc2FsdCBpbiBhIGJvd2wuIEFkZCB3YXRlciBhbmQgbWlsazsgbWl4IHVudGlsIGEgc2hhZ2d5IGRvdWdoIGZvcm1zLlxcXCIsXFxuICAgIFxcXCJLbmVhZCBhbmQgcmVzdDogS25lYWQgdW50aWwgc21vb3RoIGFuZCBlbGFzdGljIChhYm91dCA4LTEwIG1pbnMpLiBGb3JtIGludG8gYSBiYWxsIGFuZCBsZXQgcmVzdCBjb3ZlcmVkIGZvciAzMCBtaW51dGVzLlxcXCIsXFxuICAgIFxcXCJNYWtlIHRoZSBidXR0ZXIgYmxvY2s6IEZsYXR0ZW4gdGhlIGJ1dHRlciBpbnRvIGEgc3F1YXJlIChhYm91dCAxNXgxNWNtKS5cXFwiLFxcbiAgICBcXFwiTGFtaW5hdGlvbjogSW5jb3Jwb3JhdGUgdGhlIGJ1dHRlciBpbnRvIHRoZSBkb3VnaCBieSBmb2xkaW5nIGl0IG92ZXIgc2V2ZXJhbCB0aW1lcywgY2hpbGxpbmcgYmV0d2VlbiBlYWNoIGZvbGQgKGF0IGxlYXN0IDMgZm9sZHMpXFxcIixcXG4gICAgXFxcIlNoYXBlIGFuZCBwcm92ZTogUm9sbCB0aGUgZG91Z2ggaW50byBhIHJlY3RhbmdsZS4gUm9sbCB1cCB0aWdodGx5IGZyb20gb25lIGVuZC4gQ3V0IGludG8gMTIgcGllY2VzLlxcXCIsXFxuICAgIFxcXCJTaGFwZSBjcm9pc3NhbnRzOiBHZW50bHkgc3RyZXRjaCBhbmQgc2hhcGUgaW50byBjcmVzY2VudCBzaGFwZXMuXFxcIixcXG4gICAgXFxcIlNlY29uZCBwcm92ZTogUGxhY2Ugb24gYSBiYWtpbmcgc2hlZXQsIGFuZCBjb3Zlci4gTGV0IHJpc2UgaW4gYSB3YXJtIHBsYWNlIHVudGlsIGRvdWJsZWQgKDEtMS41IGhvdXJzKVxcXCIsXFxuICAgIFxcXCJCYWtlOiBQcmVoZWF0IG92ZW4gdG8gMjAwwrBDICg0MDDCsEYpLiBCcnVzaCBjcm9pc3NhbnRzIHdpdGggZWdnIHdhc2guIEJha2UgZm9yIDIwLTI1IG1pbnV0ZXMsIHVudGlsIGdvbGRlbiBicm93bi5cXFwiXFxuICBdLFxcbiAgXFxcIm5vdGVzXFxcIjogXFxcIlRoaXMgaXMgYSBiYXNpYyByZWNpcGUuIE1hbnkgdmFyaWF0aW9ucyBleGlzdCwgYW5kIGFjaGlldmluZyB0aGUgZXhhY3QgbG9vayBvZiB0aGUgaW1hZ2UgcmVxdWlyZXMgcHJhY3RpY2UgYW5kIHBvdGVudGlhbGx5IHNwZWNpZmljIHRlY2huaXF1ZXMuXFxcIlxcbn1cXG5gYGBcXG5cXG5SZW1lbWJlciB0aGF0IGJha2luZyBpcyBhbiBhcnQgYXMgd2VsbCBhcyBhIHNjaWVuY2UuICBFeHBlcmltZW50YXRpb24gaXMga2V5IHRvIGFjaGlldmluZyB0aGUgcGVyZmVjdCBjcm9pc3NhbnQhXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")